In [1]:
# MODULE IMPORTS ----

# warning settings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Stats functionality
from statsmodels.distributions.empirical_distribution import ECDF

# HDDM
import hddm

In [2]:
# Metadata
nmcmc = 300
model = 'ddm'
n_trials_per_subject = 1000
n_subjects = 5

In [3]:
# test regressors only False
# add p_outliers to the generator !
from hddm.simulators.hddm_dataset_generators import simulator_h_c

data, full_parameter_dict = simulator_h_c(data = None, 
                                          n_subjects = n_subjects,
                                          n_trials_per_subject = n_trials_per_subject,
                                          model = model,
                                          p_outlier = 0.00,
                                          conditions = None, 
                                          depends_on = None, 
                                          regression_models = ['v ~ 1 + stimulus + prevresp', 
                                                               'z ~ 1'],
                                          regression_covariates = {'prevresp': {'type': 'categorical', 'range': (0, 2)},
                                                                   'stimulus': {'type': 'categorical', 'range': (0, 2)}},
                                          group_only_regressors = False,
                                          group_only = None,
                                          fixed_at_default = None)

new round of data simulation because parameter bounds where violated


In [4]:
full_parameter_dict

{'a': 1.080707548577276,
 'a_std': 0.2556851220311648,
 't': 1.3725167904548696,
 't_std': 0.08869449638345508,
 'v_reg': OrderedDict([('v_Intercept', -0.28650372344515307),
              ('v_stimulus', -1.0335452896249364),
              ('v_prevresp', 0.4309503900813514)]),
 'v_reg_std': OrderedDict([('v_Intercept_std', 0.4381695464112052),
              ('v_stimulus_std', 0.3841358717434795),
              ('v_prevresp_std', 0.4600576989291564)]),
 'z_reg': OrderedDict([('z_Intercept', 0.37914302036710334)]),
 'z_reg_std': OrderedDict([('z_Intercept_std', 0.006847433258740332)]),
 'a_subj.0': 1.3299234456963107,
 't_subj.0': 1.4701275213079168,
 'v_Intercept': -0.28650372344515307,
 'v_stimulus': -1.0335452896249364,
 'v_prevresp': 0.4309503900813514,
 'v_Intercept_std': 0.4381695464112052,
 'v_stimulus_std': 0.3841358717434795,
 'v_prevresp_std': 0.4600576989291564,
 'v_Intercept_subj.0': -0.6563585947873427,
 'v_stimulus_subj.0': -1.3943653912935203,
 'v_prevresp_subj.0': 0.514796

In [5]:
data

,rt,response,subj_idx,prevresp,stimulus,v,a,z,t
0,1.852126,0.0,0,0.5,0.0,-0.398960,1.329923,0.387244,1.470128
1,3.462164,1.0,0,1.0,0.0,-0.141562,1.329923,0.387244,1.470128
2,3.466165,0.0,0,0.5,0.5,-1.096143,1.329923,0.387244,1.470128
3,2.466118,0.0,0,1.0,1.0,-1.535928,1.329923,0.387244,1.470128
4,2.647126,0.0,0,0.0,0.5,-1.353541,1.329923,0.387244,1.470128
...,...,...,...,...,...,...,...,...,...
4995,2.129801,0.0,4,0.0,0.5,-1.649249,1.067401,0.378312,1.575805
4996,2.525796,1.0,4,0.5,0.0,-0.431114,1.067401,0.378312,1.575805
4997,1.823805,0.0,4,0.0,1.0,-2.722316,1.067401,0.378312,1.575805
4998,2.013803,0.0,4,0.5,1.0,-2.577248,1.067401,0.378312,1.575805


In [6]:
my_model_config = hddm.model_config.model_config['ddm'].copy()

In [7]:
my_model_config['params_trans'] = [0, 0, 1, 0]
my_model_config['params_std_upper'] = [1.5, 1.0, 100, 1.0]

In [8]:
#my_model_config['slice_widths']['z_Intercept'] = 1

In [9]:
my_model_config

{'doc': 'Basic DDM. Meant for use with the LAN extension. \nNote that the boundaries here are coded as -a, and a in line with all other models meant for the LAN extension. \nTo compare model fits between standard HDDM and HDDMnn when using the DDM model, multiply the boundary (a) parameter by 2. \nWe recommend using standard HDDM if you are interested in the basic DDM, but you might want to use this for testing.',
 'params': ['v', 'a', 'z', 't'],
 'params_trans': [0, 0, 1, 0],
 'params_std_upper': [1.5, 1.0, 100, 1.0],
 'param_bounds': [[-3.0, 0.3, 0.1, 0.001], [3.0, 2.5, 0.9, 2.0]],
 'param_bounds_cnn': [[-2.5, 0.5, 0.25, 0.001], [2.5, 2.2, 0.75, 1.95]],
 'boundary': <function hddm.simulators.boundary_functions.constant(t=0)>,
 'params_default': [0.0, 1.0, 0.5, 0.001],
 'hddm_include': ['z'],
 'choices': [-1, 1],
 'slice_widths': {'v': 1.5,
  'v_std': 1,
  'a': 1,
  'a_std': 1,
  'z': 0.1,
  'z_trans': 0.2,
  't': 0.01,
  't_std': 0.15}}

In [10]:
# Make HDDM model
def z_link_func(x):
    #print(x)
    return 1 / (1 + np.exp(-(x)))
    #return x

def v_link(x):
    #print(x)
    #
    return x

#ToDo: transform the z-param so it can find the right bounds?
regr_md = [{'model': 'v ~ 1 + stimulus + prevresp', 'link_func': v_link},
           {'model': 'z ~ 1', 'link_func': z_link_func}]

# keep things as similar as possible to the usual DDM for now
hddmnn_reg = hddm.HDDMnnRegressor(data,
                                  regr_md,
                                  include = ['z'], # 'sv' is not allowed here
                                  model = 'ddm',
                                  model_config = my_model_config,
                                  informative = False,
                                  is_group_model = True, # hierarchical model
                                  group_only_regressors = True, # fit one parameter for each subject
                                  p_outlier = 0.05)

{'v': set(), 'a': set(), 'z': set(), 't': set()}
def custom_likelihood_reg(value, v, a, z, t, reg_outcomes, p_outlier=0, w_outlier=0.1, **kwargs):
    params = locals()
    size = int(value.shape[0])
    data = np.zeros(((size, 6)), dtype=np.float32)
    param_links = {'v': set(), 'a': set(), 'z': set(), 't': set()}
    data[:, 4:] = np.stack([np.absolute(value["rt"]).astype(np.float32), value["response"].astype(np.float32)], axis=1)
    cnt=0
    for tmp_str in ['v', 'a', 'z', 't']:
        if tmp_str in reg_outcomes:
            data[:, cnt] = params[tmp_str].loc[value["rt"].index].values
            for linked_indirect_regressor in param_links[tmp_str]:
                data[:, cnt] = data[:, cnt] + params[linked_indirect_regressor].loc[value["rt"].index].values
            if (data[:, cnt].min() < [-3.0, 0.3, 0.1, 0.001][cnt]) or (data[:, cnt].max() > [3.0, 2.5, 0.9, 2.0][cnt]):
                warnings.warn("boundary violation of regressor part")
                return -np.inf
    

In [11]:
hddm_reg_standard = hddm.HDDMRegressor(data,
                                         regr_md,
                                         include = ['z'], # 'sv' is not allowed here
                                         informative = False,
                                         is_group_model = True, # hierarchical model
                                         group_only_regressors = False, # fit one parameter for each subject
                                         p_outlier = 0.05)

No model attribute --> setting up standard HDDM
Set model to ddm
Reg Model:
{'outcome': 'v', 'model': ' 1 + stimulus + prevresp', 'params': ['v_Intercept', 'v_stimulus', 'v_prevresp'], 'link_func': <function v_link at 0x141b1be60>}
v
trans:  0
is_indirect_regressor 0
link_is_identity True
Reg Model:
{'outcome': 'z', 'model': ' 1', 'params': ['z_Intercept'], 'link_func': <function z_link_func at 0x141db0d40>}
z
trans:  1
is_indirect_regressor 0
link_is_identity False


In [12]:
# Sample
hddmnn_reg.sample(nmcmc, burn = 100)

 [-----------------100%-----------------] 301 of 300 complete in 159.6 sec

In [36]:
# Sample
hddm_reg_standard.sample(nmcmc, burn = 100)

 [---               8%                  ] 24 of 300 complete in 9.0 secHalting at iteration  23  of  300
Could not generate output statistics for t_subj.0
Could not generate output statistics for v_stimulus_subj.0
Could not generate output statistics for v_Intercept_subj.1
Could not generate output statistics for v_Intercept_subj.2
Could not generate output statistics for z_Intercept_subj.1
Could not generate output statistics for z_Intercept_trans
Could not generate output statistics for t_subj.2
Could not generate output statistics for v_stimulus_subj.3
Could not generate output statistics for v_prevresp_std
Could not generate output statistics for t_subj.1
Could not generate output statistics for v_prevresp_subj.1
Could not generate output statistics for v_prevresp_subj.0
Could not generate output statistics for v_Intercept_subj.0
Could not generate output statistics for z_Intercept
Could not generate output statistics for t_std
Could not generate output statistics for v_stimulus_su

TypeError: 'NoneType' object is not subscriptable

In [9]:
hddmnn_reg.model_config['params_trans'] = [0, 0, 0, 0]

In [11]:
hddm.model_config.model_config

{'ddm_vanilla': {'doc': 'Model used internally for simulation purposes. Do NOT use with the LAN extension.',
  'params': ['v', 'a', 'z', 't'],
  'params_trans': [0, 0, 1, 0],
  'params_std_upper': [1.5, 1.0, None, 1.0],
  'param_bounds': [[-5.0, 0.1, 0.05, 0], [5.0, 5.0, 0.95, 3.0]],
  'boundary': <function hddm.simulators.boundary_functions.constant(t=0)>,
  'params_default': [0.0, 2.0, 0.5, 0],
  'hddm_include': ['z'],
  'choices': [0, 1],
  'slice_widths': {'v': 1.5,
   'v_std': 1,
   'a': 1,
   'a_std': 1,
   'z': 0.1,
   'z_trans': 0.2,
   't': 0.01,
   't_std': 0.15}},
 'full_ddm_vanilla': {'doc': 'Model used internally for simulation purposes. Do NOT use with the LAN extension.',
  'params': ['v', 'a', 'z', 't', 'sz', 'sv', 'st'],
  'params_trans': [0, 0, 1, 0, 0, 0, 0],
  'params_std_upper': [1.5, 1.0, None, 1.0, 0.1, 0.5, 0.1],
  'param_bounds': [[-5.0, 0.1, 0.3, 0.25, 0, 0, 0],
   [5.0, 5.0, 0.7, 2.25, 0.25, 4.0, 0.25]],
  'boundary': <function hddm.simulators.boundary_functi

In [12]:
def my_fun():
    c = 1
    return


In [21]:
my_fun

<function __main__.my_fun()>